### 安装与使用
安装python 驱动器： pip install psycopg2        

启动server：pg_ctlcluster 11 main start      

su - postgres

psql

\password postgres 设置密码

In [1]:
import psycopg2

In [4]:
conn=psycopg2.connect(database="postgres",user="postgres",password="123456",host="localhost",port="5432")
cur=conn.cursor() #创建指针对象

In [5]:
# 创建表
cur.execute("CREATE TABLE student(id integer,name varchar,sex varchar);")

In [6]:
#插入数据
cur.execute("INSERT INTO student(id,name,sex)VALUES(%s,%s,%s)",(1,'Aspirin','M'))
cur.execute("INSERT INTO student(id,name,sex)VALUES(%s,%s,%s)",(2,'Taxol','F'))
cur.execute("INSERT INTO student(id,name,sex)VALUES(%s,%s,%s)",(3,'Dixheral','M'))

In [7]:
# 获取结果
cur.execute('SELECT * FROM student')
results=cur.fetchall()
print (results)

[(1, 'Aspirin', 'M'), (2, 'Taxol', 'F'), (3, 'Dixheral', 'M')]


In [8]:
# 关闭连接
conn.commit()
cur.close()
conn.close()

### 创建testdb 与 dbuser1 
Create User dbuser1 WITH PASSWORD '123456';

Create DATABASE testdb OWNER dbuser1;

GRANT ALL PRIVILEGES ON DATABASE testdb TO dbuser1;


In [39]:
conn=psycopg2.connect(database="testdb",user="dbuser1",password="123456",host="localhost",port="5432")
cur=conn.cursor()

In [40]:
cur.execute('select version();')
results=cur.fetchall()
print (results)

[('PostgreSQL 11.5 (Ubuntu 11.5-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0, 64-bit',)]


In [43]:
# 创建表, 注意多行字符串
creates = '''drop table if exists COMPANY;
CREATE TABLE COMPANY 
       (ID INT PRIMARY KEY     NOT NULL,
       NAME           TEXT    NOT NULL,
       AGE            INT     NOT NULL,
       ADDRESS        VARCHAR(50),
       SALARY         REAL);
'''
creates

'drop table if exists COMPANY;\nCREATE TABLE COMPANY \n       (ID INT PRIMARY KEY     NOT NULL,\n       NAME           TEXT    NOT NULL,\n       AGE            INT     NOT NULL,\n       ADDRESS        VARCHAR(50),\n       SALARY         REAL);\n'

In [46]:
cur.execute(creates)

In [47]:
cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) VALUES (1, 'Paul', 32, 'California', 20000.00 );")

In [48]:
cur.execute("SELECT id, name, address, salary  from COMPANY")
results=cur.fetchall()
print (results)

[(1, 'Paul', 'California', 20000.0)]


In [45]:
conn.commit()

In [31]:
conn.commit()
cur.close()
conn.close()

### API 
```
psycopg2.Connect()参数说明
dbname(str):  数据库名称
user(str):      用户名      
password(str):  密码
host(str):      MySQL服务器地址      
port(int):      MySQL服务器端口号      

connection对象支持的方法      
cursor()        使用该连接创建并返回游标
commit()        提交当前事务
rollback()      回滚当前事务
close()         关闭连接

cursor对象支持的方法
execute(op)     执行一个数据库的查询命令
fetchone()      取得结果集的下一行
fetchmany(size) 获取结果集的下几行
fetchall()      获取结果集中的所有行
rowcount()      返回数据条数或影响行数
close()         关闭游标对象
```

python Db 接口对比：   
https://wiki.python.org/moin/DbApiCheatSheet

### 操作fdw 

In [1]:
import psycopg2

In [2]:
conn=psycopg2.connect(database="postgres",user="postgres",password="123456",host="localhost",port="5432")
cur=conn.cursor()

In [12]:
# 防止重复生成，使用 Cascade; 
createfdw = '''
drop extension file_fdw CASCADE;
CREATE EXTENSION file_fdw;
CREATE SERVER server_csv FOREIGN DATA WRAPPER file_fdw;
'''

In [15]:
cur.execute(createfdw)

In [16]:
# 查看磁盘使用量
dfsql = '''
drop FOREIGN table if EXISTS disk_free;
    CREATE FOREIGN TABLE disk_free (
file_system TEXT,
size varchar(50),
used  varchar(50),
avail  varchar(50),
capacity TEXT,
mounted_on TEXT
) SERVER server_csv OPTIONS (PROGRAM
$$
df -h| awk '{print $1,$2,$3,$4,$5,$6}' OFS='\037'
$$
, FORMAT 'csv', HEADER 'TRUE', DELIMITER E'\037');
'''

In [17]:
cur.execute(dfsql)

In [18]:
# 查看结果
cur.execute("select * from disk_free")
results=cur.fetchall()
print (results)

[('rootfs', '116G', '64G', '52G', '56%', '/'), ('none', '116G', '64G', '52G', '56%', '/dev'), ('none', '116G', '64G', '52G', '56%', '/run'), ('none', '116G', '64G', '52G', '56%', '/run/lock'), ('none', '116G', '64G', '52G', '56%', '/run/shm'), ('none', '116G', '64G', '52G', '56%', '/run/user'), ('cgroup', '116G', '64G', '52G', '56%', '/sys/fs/cgroup'), ('C:\\', '116G', '64G', '52G', '56%', '/mnt/c'), ('D:\\', '56G', '17G', '40G', '30%', '/mnt/d'), ('E:\\', '67G', '15G', '52G', '23%', '/mnt/e'), ('F:\\', '466G', '243G', '224G', '52%', '/mnt/f')]


In [36]:
def printsql(cur, length = 10):
    # 获取结果集的每一行
    rows = cur.fetchall()
    # 获取所有字段名，tuple
    all_fields = cur.description
    
    
    # 首先打印字段名
    field_messages = []
    for i in range(len(all_fields)):
        # 格式化输出结果，len参数是各列的显示宽度，可以指定常量，也可自定义函数进行获取。
        field_messages.append("{str:<{len}}".format(str=str(all_fields[i][0]), len=length))
    
    field_message = "".join(field_messages)
    print(field_message)
    
    # 然后逐行打印结果集
    for row in rows:
        row_messages = []
        for j in range(len(row)):
            # 格式化结果集
            row_messages.append("{str:<{len}}".format(str=str(row[j]), len=length))
        row_message = "".join(row_messages)
        print(row_message)

In [38]:
cur.execute("select * from disk_free")
printsql(cur, 12)

file_system size        used        avail       capacity    mounted_on  
rootfs      116G        64G         52G         56%         /           
none        116G        64G         52G         56%         /dev        
none        116G        64G         52G         56%         /run        
none        116G        64G         52G         56%         /run/lock   
none        116G        64G         52G         56%         /run/shm    
none        116G        64G         52G         56%         /run/user   
cgroup      116G        64G         52G         56%         /sys/fs/cgroup
C:\         116G        64G         52G         56%         /mnt/c      
D:\         56G         17G         40G         30%         /mnt/d      
E:\         67G         15G         52G         23%         /mnt/e      
F:\         466G        243G        224G        52%         /mnt/f      


In [34]:
# ^, <, > 分别是居中、左对齐、右对齐，后面带宽度，默认是用空格填充。
"{str:<7} ".format(str = 'hello')

'hello   '

In [50]:
# python 可以允许出notebook输入
a = input()
print(a)

good
good
